<a href="https://colab.research.google.com/github/joaopcastro710/Proj_AI/blob/main/Project%202/Proj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello")
print("bye")

import pandas as pd

#TPU google collab

train_df = pd.read_csv("Dataset2.csv")

# Preview data
print(train_df.head())
print(train_df.info())
print(train_df.describe())

#treino: Dataset2.csv
#treino: merged_dataset.csv
#teste: Dataset1.csv

hello
bye
<module 'posixpath' (frozen)>


NameError: name 'train_df' is not defined